In [ ]:
from gym import envs
all_envs = envs.registry.all()
env_ids = [env_spec.id for env_spec in all_envs]
print(sorted(env_ids))

In [ ]:
import numpy as np
class ObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete), \
            "Should only be used to wrap Discrete envs."
        self.n = self.observation_space.n
        self.observation_space = gym.spaces.Box(0, 1, (self.n,))
    
    def observation(self, obs):
        new_obs = np.zeros(self.n)
        new_obs[obs] = 1
        return new_obs

In [ ]:


import gym
env = ObservationWrapper(gym.make('Taxi-v3'))
#env = gym.make('Taxi-v3')
print(env.action_space)
print(env.observation_space)



env = gym.make('CartPole-v0')
print(env.action_space)
print(env.observation_space)





In [1]:
import numpy as np
from src.jss_lite.jss_lite import jss_lite
t=np.empty((4,1),dtype=object)
import time
env=jss_lite()


state=env.reset()
for i in range(150):
    legal_action=env.get_legal_actions(state)
    action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
    if action ==10:
        print(env.current_machines_status)
        env.render()
        state, reward, done, info=env.step(action)
        print(env.current_machines_status)
        env.render()
    else:
        state, reward, done, info=env.step(action)
    print(action)
    #print(env.count_finished_tasks_job_matrix)
    #print(env.count_finished_tasks_machine_matrix)
    #print(env.current_timestep)
    #env.render()
    if done==True:
        print(f"finished! no legal actions possible after {i} steps")
        break
    


/Users/felix/sciebo/masterarbeit/progra/model-based_rl/src/jss_lite/jss_lite.py:346: RuntimeWarning: invalid value encountered in double_scalars
  return round(value/max_value,4)


8
7
2
5
6
4
1
[[ 8. 94.  0.]
 [ 5. 99.  0.]
 [nan  0.  0.]
 [ 4. 69.  0.]
 [ 7. 98.  0.]
 [ 1. 72.  0.]
 [nan  0.  0.]
 [ 6. 50.  0.]
 [nan  0.  0.]
 [ 2. 83.  0.]]


new time is 0
[[nan 94.  0.]
 [nan 99.  0.]
 [nan  0.  0.]
 [nan 69.  0.]
 [nan 98.  0.]
 [nan 72.  0.]
 [nan  0.  0.]
 [nan 50.  0.]
 [nan  0.  0.]
 [nan 83.  0.]]


/Users/felix/sciebo/masterarbeit/progra/model-based_rl/src/jss_lite/jss_lite.py:346: RuntimeWarning:

invalid value encountered in double_scalars



10
0
8
3
2
6
1
[[nan 94.  0.]
 [ 6. 86.  0.]
 [nan  0.  0.]
 [ 1. 50.  0.]
 [ 0. 88.  0.]
 [nan 72.  0.]
 [ 8. 71.  0.]
 [ 3. 94.  0.]
 [ 2. 61.  0.]
 [nan 83.  0.]]


...

new time is 94
[[nan 89. 34.]
 [nan 80. 77.]
 [nan 61. 23.]
 [nan 51. 85.]
 [nan 74. 88.]
 [nan 60. 20.]
 [nan 73. 77.]
 [nan 88. 69.]
 [nan 61. 85.]
 [nan 74.  8.]]


10
8
9
0
3
4
1
[[ 3. 76. 34.]
 [nan 80. 77.]
 [ 0. 89. 23.]
 [nan 51. 85.]
 [nan 74. 88.]
 [ 8. 58. 20.]
 [ 4. 95. 77.]
 [ 9. 56. 69.]
 [nan 61. 85.]
 [ 1. 63.  8.]]


/Users/felix/sciebo/masterarbeit/progra/model-based_rl/src/jss_lite/jss_lite.py:346: RuntimeWarning:

invalid value encountered in double_scalars



new time is 98


IndexError: index 10 is out of bounds for axis 0 with size 10

In [3]:
env.render()
print(env.production_list)

[[(8, 0, 0, 94) (2, 2, 50, 133) (9, 1, 61, 120) None (1, 6, 83, 175)
  (3, 7, 94, 170) None None None None]
 [(5, 0, 0, 99) (6, 1, 0, 86) None (3, 2, 69, 130) (9, 2, 71, 153)
  (8, 4, 72, 138) (0, 4, 83, 150) (1, 7, 86, 168) None None]
 [None None (3, 1, 50, 118) None None (1, 4, 71, 165) None
  (8, 5, 83, 173) (0, 5, 94, 183) None]
 [(1, 1, 0, 50) (9, 0, 50, 100) (6, 3, 69, 165) (7, 2, 72, 154)
  (3, 4, 83, 137) None None None None None]
 [(7, 0, 0, 98) (4, 1, 50, 138) (6, 2, 61, 158) (1, 3, 69, 144)
  (3, 3, 72, 171) (8, 6, 86, 162) None None None None]
 [(1, 0, 0, 72) (2, 5, 71, 156) (5, 2, 72, 136) None None (8, 7, 94, 152)
  None None None None]
 [None (8, 1, 0, 71) (0, 2, 61, 155) (2, 4, 69, 133) (7, 1, 71, 144)
  (5, 3, 83, 149) (3, 5, 86, 161) (4, 6, 94, 189) None None]
 [(6, 0, 0, 50) (8, 3, 69, 154) (2, 6, 72, 150) (9, 4, 94, 150) None None
  None None None None]
 [None (2, 1, 0, 61) (0, 1, 50, 118) (4, 3, 69, 164) (1, 5, 72, 138)
  (6, 5, 83, 180) (5, 4, 86, 166) (9, 3, 88, 

In [ ]:
import ale_py as ale
ale.importroms('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/roms/Space Invaders.bin')
import gym
env = gym.make('SpaceInvaders-v4')
env.reset()
env.render()


In [2]:
import JSSEnv
import src.jss_graph_env.disjunctive_graph_jss_env as jss_env
import src.jsp_instance_parser 
import time
import gym
import pandas as pd
import random
import numpy as np
from src.jss_lite.jss_lite import jss_lite

index=['klagenfurt','alex','lite']
df=pd.DataFrame(np.zeros((3,5)),columns=['setuptime','resettime','10xactiontime','col4','col5'],index=index)
#df=pd.DataFrame
#df.set_index('index')
for i in range(100):

    def timer(innerfun):
        tmp_t=time.time()
        tmp_r=innerfun()
        return time.time()-tmp_t,tmp_r

    tmp_t=time.time()
    env2=gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/abz8.txt'})
    t_1=(time.time()-tmp_t)
    df['setuptime']['klagenfurt']+=t_1/100
    tmp_t=time.time()
    env_3=jss_lite()
    t_1=time.time()-tmp_t
    df['setuptime']['lite']+=t_1/100




    tmp_t=time.time()
    path='resources/jsp_instances/standard/abz8.txt'
    curr_instance=src.jsp_instance_parser.parse_jps_standard_specification(path)
    res,std_matrix=curr_instance
    env_alex = jss_env.DisjunctiveGraphJssEnv(res,default_visualisations='gantt_console')
    t_2=(time.time()-tmp_t)
    df['setuptime']['alex']+=t_2/100
    tmp_t=time.time()
    t=env2.reset()
    t_1=(time.time()-tmp_t)
    df['resettime']['klagenfurt']+=t_1/100
    tmp_t=time.time()
    t2=env_alex.reset()
    t_2=(time.time()-tmp_t)
    df['resettime']['alex']+=t_2/100
    tmp_t=time.time()
    t=env_3.reset()
    t_1=(time.time()-tmp_t)
    df['resettime']['lite']+=t_1/100


    state=env2.reset()
    for i in range(5):
        legal_action=state['action_mask']
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        tmp_t=time.time()
        state, reward, done, info=env2.step(action)
        t_1=(time.time()-tmp_t)
        df['10xactiontime']['klagenfurt']+=t_1/5

        if done==True:
            print(f"finished! no legal actions possible after {i} steps")
            break
    state=env_3.reset()
    for i in range(5):
        legal_action=env_3.get_legal_actions(state)
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        tmp_t=time.time()
        state, reward, done, info=env_3.step(action)
        t_1=(time.time()-tmp_t)
        df['10xactiontime']['lite']+=t_1/5

        if done==True:
            print(f"finished! no legal actions possible after {i} steps")
            break



    action_list=np.arange(0,env_alex.n_jobs*env_alex.n_machines)
    for i in range(5):
        action_mask=env_alex.valid_action_mask()
        actions=action_list[action_mask]
        next_action=random.choice(actions)
        #next_action=actions[action_mask[0]]
        #print(next_action)
        tmp_t=time.time()
        state,reward,done,info =env_alex.step(next_action)
        t_2=(time.time()-tmp_t)
        df['10xactiontime']['alex']+=t_2/5
        if done == True:
            print(f"finished after {i} steps")
            break
        #env.render()






[12:22:01] INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'

           INFO     instance appears to contain 20 jobs, 15 machines

/Users/felix/sciebo/masterarbeit/progra/model-based_rl/src/jss_lite/jss_lite.py:318: RuntimeWarning: invalid value encountered in double_scalars
  return round(value/max_value,4)


           INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'

           INFO     instance appears to contain 20 jobs, 15 machines

...

           INFO     instance appears to contain 20 jobs, 15 machines

           INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'

           INFO     instance appears to contain 20 jobs, 15 machines

[12:22:06] INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'

           INFO     instance appears to contain 20 jobs, 15 machines

In [25]:
import resource
import plotly.express as px
import pandas as pd
from datetime import datetime, timedelta
dt = datetime(2022, 1, 1, 0, 0, 0)
#stores quatuple(job,task,start_time,finish_time) 
time_now=time.time()
production_list=env_3.production_list
def production_to_dict(input,i,j):
    return(dict(Task=f"job_{input[0]}", Start=dt + timedelta(seconds=int(input[2])),Finish=dt + timedelta(seconds=int(input[3])),Resource=f"machine_{i}"))
df=pd.DataFrame()
liste=[]
for i in range(10):
    for j in range(10):
        if production_list[i,j] is not None:
            liste.append(production_to_dict(production_list[i,j],i,j))

df_2=pd.DataFrame(liste)
fig = px.timeline(df_2, x_start="Start", x_end="Finish", y="Task", color="Resource")
fig.show()

df = pd.DataFrame([
    dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30', Resource="Max")
])

#fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource", color="Resource")
#fig.show()

2022-01-01 00:00:24
